<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/bigtweet_ask_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
base_dir = 'medical-concept-normalization'

In [ ]:
os.mkdir(base_dir)

In [ ]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [ ]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [ ]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [ ]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [ ]:
path = Path('medical-concept-normalization/data_collection')

In [ ]:
# training data set

train_csv = path/'AskAPatient.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [ ]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heels hurt
2,4,black and blue marks
3,5,fast and slow heartbeat
4,5,increased heart rate


In [ ]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-6.test.csv",header=None)
test.head()

,0,1
0,2,left heel pain
1,5,tachycardia
2,8,ARTHRITIS
3,8,arthrites
4,8,arthritis


In [ ]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'bigtweet_ask.csv')

In [ ]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [ ]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [ ]:
data_clas.add_test(test,label=0)


In [ ]:
data_lm.save()
data_clas.save()

In [ ]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'lipitor',
 'in',
 'of',
 'muscle',
 'to',
 'the',
 'severe',
 'and',
 'my',
 'myalgia',
 'rt',
 'fatigue',
 'a',
 'lower',
 'loss',
 'cramp',
 'limb',
 'weakness',
 'for',
 'stomach',
 'disease',
 'depression',
 '-',
 'back',
 'on',
 'cramps',
 'gas',
 'is',
 'upper',
 'joint',
 'memory',
 'leg',
 'you',
 'legs',
 'pains',
 'i',
 'headache',
 'with',
 'unable',
 'shoulder',
 'arthralgia',
 'knee',
 'neck',
 'excessive',
 'foot',
 'aches',
 'amp',
 'gastrointestinal',
 'hip',
 'numbness',
 'arthritis',
 'not',
 'all',
 'insomnia',
 '/',
 'like',
 'gluten',
 'arthrotec',
 'tired',
 'asthenia',
 'nausea',
 'bleeding',
 'walk',
 'abdominal',
 'heart',
 'feeling',
 'walking',
 'problems',
 'dizziness',
 'difficulty',
 'feet',
 'swelling',
 'chest',
 'muscles',
 'hand',
 'hair',
 'backache',
 'at',
 'ache',
 'extreme',
 'burning',
 'vision',
 'could',
 'your',
 'zocor',
 'stiffness',
 'out',


In [ ]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.260208,2.750072,0.551292,19:32


In [ ]:
# learn.fit_one_cycle(1, 1e-2)

In [ ]:
# learn.fit_one_cycle(1, 1e-3)

In [ ]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.841858,2.697870,0.555993,19:28


In [ ]:
learn.save_encoder('bigtweet_ask_ask_fold6_first')

In [ ]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_ask_ask_fold6_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.054873,3.783264,0.340909,01:10


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.001810,3.036321,0.475936,01:17


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.431617,2.620968,0.532086,03:13


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.063375,2.184874,0.604278,03:17
1,1.762692,1.993476,0.640374,03:30


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.586507,1.816137,0.664438,03:08
1,1.372575,1.696053,0.692513,03:27


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.249641,1.564088,0.713904,03:21
1,1.118466,1.481499,0.727273,03:11


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.016417,1.424529,0.741979,03:13
1,0.877735,1.349625,0.758021,03:16


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.839912,1.264961,0.768717,03:17
1,0.744904,1.228769,0.768717,03:22


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.663759,1.261686,0.774064,03:10
1,0.677763,1.175763,0.787433,03:19
2,0.596936,1.089170,0.795455,03:25
3,0.521178,1.086093,0.794118,03:12


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.506872,1.127408,0.799465,03:10
1,0.554201,1.077550,0.796791,03:30
2,0.449205,1.054214,0.808824,03:12
3,0.435569,1.048589,0.806150,03:28


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.370084,1.082636,0.804813,03:25
1,0.416731,1.151476,0.792781,03:16
2,0.392349,1.035563,0.812834,03:12
3,0.324086,1.019267,0.807487,03:27


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.298475,1.122179,0.804813,03:25
1,0.378408,1.087662,0.802139,03:14
2,0.332743,1.077229,0.816845,03:15
3,0.278202,1.067940,0.815508,03:17


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.282881,1.108994,0.808824,03:26
1,0.307663,1.151385,0.816845,03:14
2,0.295036,1.061775,0.818182,03:09
3,0.213889,1.058665,0.820856,03:21


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.259713,1.114189,0.816845,03:14
1,0.272140,1.094696,0.827540,03:11
2,0.241592,1.017878,0.827540,03:22
3,0.215329,1.058790,0.823529,03:15


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.213891,1.118280,0.815508,03:06
1,0.265767,1.177649,0.815508,03:11
2,0.249940,1.087508,0.820856,03:06
3,0.208576,1.136600,0.823529,03:29


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.203360,1.116158,0.822192,03:30
1,0.201633,1.161157,0.828877,03:07


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.240867,1.134627,0.828877,03:09
1,0.208614,1.115148,0.826203,03:13


In [ ]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.194405,1.190693,0.818182,03:12


In [ ]:
# save the best model

learn.save_encoder('bigtweet_ask_ask_mixed_fold6')

# Part three: Predict on the test dataset

In [ ]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

In [ ]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)